In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import cudf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import matplotlib.pyplot as plt
import pickle
import os
# from sklearn.cluster import KMeans
# from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import StandardScaler
import time
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
print(np.__version__)
print(pd.__version__)

In [ ]:
import riiideducation
# import resource

In [ ]:
# from multiprocessing import Pool, TimeoutError

In [ ]:
# np.random.seed(42) # generating random see
# tf.random.set_seed(42) # setting random seed
inputDataFolder ='/kaggle/input/prediction-with-score/'

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def printVariableSize(li):
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in li.items()),
                             key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

# Load Data

In [ ]:
%%time
outfile = open(inputDataFolder+'user_history_group_dict.pkl','rb')
train = pickle.load(outfile)
outfile.close()
# train = cudf.from_pandas(train)
train_keys = train.keys()

In [ ]:
train[115].info()

In [ ]:
# %%time
outfile = open(inputDataFolder+'user_score.pkl','rb')
user_score = pickle.load(outfile)
outfile.close()

In [ ]:
user_score= user_score.astype('float32')
user_score.dtype

In [ ]:
# %%time
outfile = open(inputDataFolder+'question_weights.pkl','rb')
question_weights = pickle.load(outfile)
outfile.close()

In [ ]:
# %%time
outfile = open(inputDataFolder+'content_cat.pkl','rb')
content_df = pickle.load(outfile).drop(columns=['new_content_id'])
outfile.close()
# content_df = cudf.from_pandas(content_df)

In [ ]:
content_df=content_df.astype('float32')
content_df.info()

In [ ]:
# %%time
outfile = open(inputDataFolder+'sc_timestamp.pkl','rb')
sc_timestamp = pickle.load(outfile)
outfile.close()

In [ ]:
# %%time
outfile = open(inputDataFolder+'new_content_id_index_map.pkl','rb')
new_content_id_index_map = pickle.load(outfile)
outfile.close()
# new_content_id_index_map = pd.from_pandas(new_content_id_index_map)

In [ ]:
new_content_id_index_map_dict = { new_content_id_index_map.iloc[i,1]:new_content_id_index_map.iloc[i,0] for i in range(len(new_content_id_index_map))}
new_content_id_index_map_dict_keys = new_content_id_index_map_dict.keys()
# new_content_id_index_map_dict

In [ ]:
# %%time
outfile = open(inputDataFolder+'valid_df.pkl','rb')
valid_df = pickle.load(outfile)
outfile.close()

In [ ]:
printVariableSize(globals())

# Functions

In [ ]:
def formatData(testData):

    testData['prior_question_had_explanation']=testData['prior_question_had_explanation'].fillna(False)
    testData['prior_question_had_explanation']=testData['prior_question_had_explanation'].astype(int)

    testData['new_content_id'] = testData['content_type_id'].astype(str)+'_'+testData['content_id'].astype(str)

    testData.loc[:,['timestamp','prior_question_elapsed_time']] = sc_timestamp.transform(testData.loc[:,['timestamp','prior_question_elapsed_time']])

    testData = testData[['user_id','timestamp', 'new_content_id', 'content_type_id', 
                  'prior_question_had_explanation', 'answered_correctly']]
    
    testData=testData.astype({'timestamp':'float32', 'prior_question_had_explanation': 'float32','content_type_id':'float32'})

    for i in range(len(testData)):
        if testData.iloc[i,2] in new_content_id_index_map_dict_keys:
            testData.iloc[i,2] = new_content_id_index_map_dict[testData.iloc[i,2]]
        else:
            testData.iloc[i,2] = 0


    return testData

In [ ]:
new_user_score = .5
depthOfHistory = 5

In [ ]:
def testData_preProcessing(testData):

    testData_np = testData.to_numpy()
    X_submit = np.empty((len(testData), 207))
    X_submit[:,:6] = testData_np
    for i in np.arange(len(X_submit)):
        if X_submit[i,0] in user_score.index:
            X_submit[i,6]= user_score[X_submit[i,0]]  
        else:
            X_submit[i,6]= new_user_score
        X_submit[i,7:]= content_df.loc[X_submit[i,2],:]

    X_submit=np.delete(X_submit, [2,5],1)

    testData_filtered=[]

    for i in X_submit:
        
        if i[0] in train_keys:

            lenOfHist = len(train[i[0]])
            if lenOfHist >= depthOfHistory:
                heightOfArray = depthOfHistory
            else:
                heightOfArray = lenOfHist

            tempHist = np.zeros((heightOfArray, 207))

            tempHist[:,:7] = train[i[0]].tail(depthOfHistory).astype({'timestamp':'float32', 'prior_question_had_explanation': 'float32','content_type_id':'float32'}).to_numpy()
               
            tempHist[:,7:]= content_df.loc[tempHist[:,6],:]

            tempHist=np.delete(tempHist, [4,6],1)

            temp_all = np.append(tempHist,[i], axis=0)


        else:
            temp_all=[i]
        
        temp_all=np.delete(temp_all, 0,1)
        testData_filtered.append([temp_all])

  
    return testData_filtered


# Generator

In [ ]:

def genTestData1(data):

    data.reset_index(drop=True,inplace=True)

    data = testData_preProcessing(data)

    for g in data:

        yield tf.convert_to_tensor(g, dtype=np.float32)





In [ ]:
def genTestData(data):
    while True:
        
        yield tf.convert_to_tensor(testData_filtered_dummy[0], dtype=np.float32)

In [ ]:
def preProcessData(data):

    data.reset_index(drop=True,inplace=True)
    return testData_preProcessing(data)

    

# Model

In [ ]:
model = keras.models.load_model(inputDataFolder+'latest_model.h5')
# model = keras.models.load_model(inputDataFolder+'best_model_20.h5')
model.call = tf.function(model.call, experimental_relax_shapes=True)
model.summary()

# Prediction

In [ ]:
env = riiideducation.make_env()
iter_test = env.iter_test()
# iter_test = valid_gen()

In [ ]:

for (test_df, sample_prediction_df) in iter_test:

    test_df.reset_index(drop=True,inplace=True)

    test_df['answered_correctly']=-1
    if len(test_df)>0:

        test_df_mod =formatData(test_df)


        X_pred= test_df_mod[test_df_mod['content_type_id'] == 0]
        len_X_pred = len(X_pred)

        if len_X_pred >0:


            data = preProcessData(X_pred)

            y_pred =[]

            for d in data:

                train_dataset=tf.convert_to_tensor(d , dtype=np.float32)
                y_pred_temp = model(train_dataset)
                
                y_pred.append(y_pred_temp[0][0])

            y_pred = np.asarray(y_pred)


            test_df.loc[test_df['content_type_id'] == 0,'answered_correctly'] = np.nan_to_num(y_pred,nan=0.9,posinf=.9,neginf=0.1)

            env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

        else :
            env.predict(np.array([], dtype = np.float32))

    else:
        env.predict(np.array([], dtype = np.float32))
